# American Sign Language (ASL) Final Project W251 YOLOv5 Custom Dataset
---
 
## Set up on Nvidia Jetson NX

- Using Docker Image Inference which uses the base image nvcr.io/nvidia/l4t-pytorch:r32.6.1-pth1.9-py3 from the Dockerfile.yolov5
- Building Docker image

```bash
docker build -t yolov5 -f Dockerfile.yolov5 . 
```
- Run the container and mounting data in the /app/ folder

```bash
docker run -ti --rm --runtime nvidia  --device /dev/video0 --network host --privileged -e DISPLAY=$DISPLAY -v /data:/app/data yolov5
```
- Run the following line before spinning up Jupyter to update Jupyter config

```bash
export LD_PRELOAD=/usr/lib/aarch64-linux-gnu/libgomp.so.1
```


#### 1. Download Yolov5 Repository & Data

In [ ]:
!pwd

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5

In [ ]:
import torch

from IPython.display import Image, clear_output  # to display images

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

#### 2. Make sure to download the dataset from Roboflow as YOLO v5 PyTorch

In [ ]:
!mkdir ./data/asl6

In [ ]:
%cd ./data/asl6

In [ ]:
!curl -L "https://app.roboflow.com/ds/KnljOtVDPM?key=37TDe9XWw3" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
%cat data.yaml

In [ ]:
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%cd ../..

In [ ]:
print(num_classes)

In [ ]:
%%writetemplate models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
!ls

## Inference with Jetson NX - 32FP

In [ ]:
!pwd

In [ ]:
torch.cuda.empty_cache()
torch.cuda.is_available()

In [ ]:
# Testing with ASL model
!python3 val.py --weights ./weights/yolov5_300_epochs_best_model_all_images.pt --data ./data/asl6/data.yaml

In [ ]:
# Running Inference with live camera - PF32
!python3 detect.py --source 0 --weights ./weights/yolov5_300_epochs_best_model_all_images.pt --conf 0.4 --data ./data/asl6/data.yaml

## Inference with Jetson NX - Tensort RT

In [ ]:
# Export to tensortrt models
!python3 export.py --weights ./weights/yolov5_300_epochs_best_model_all_images.pt  --include torchscript engine --device 0

In [ ]:
# Tensort RT model
!python3 detect.py --source 0 --weights ./weights/yolov5_best_300_epochs.engine --conf 0.4 --data ./data/asl6/data.yaml

## Inference with Jetson NX - Torchscript

In [ ]:
# Torchscript model
!python3 detect.py --source 0 --weights ./weights/yolov5_best_300_epochs.torchscript --conf 0.4 --data ./data/asl6/data.yaml

## Inference with Jetson NX - ONNX

In [ ]:
#ONNX model with OpenCV DNN
!python3 detect.py --source 0 --weights ./weights/yolov5_best_300_epochs.onnx --conf 0.4 --data ./data/asl6/data.yaml